This encodes the Hamiltonian of the helium hydride molecule HeH$^+$. The energy $\langle H \rangle$ is measured as follows:
$$
\langle H \rangle = p(00) + p(01) - p(10) - p(11).
$$
The lowest energy state is then found using a loop of quantum circuit executions and classical optimization.

In [24]:
import numpy as np

from quantuminspire.credentials import enable_account, get_token_authentication
from quantuminspire.api import QuantumInspireAPI

SEED = 0xbeef

enable_account('6ea0bc8530ed4c97073aff535b5b52fd617cc6ed')
auth = get_token_authentication()
server_url = r'https://api.quantum-inspire.com'
qi = QuantumInspireAPI(server_url, auth)

def estimate_hamiltonian(params, shots=1024, backend='QX single-node simulator'):
    print(f'Running quantum circuit with params {params}')
    
    qasm = f'''version 1.0
qubits 2

.ansatz
{{Rx q[0], {params[0]} | Rx q[1], {params[1]}}}
{{Rz q[0], {params[2]} | Rz q[1], {params[3]}}}

H q[0]
CNOT q[0], q[1]
H q[1]
CNOT q[1], q[0]

{{Rz q[0], {params[4]} | Rz q[1], {params[5]}}}
{{Rx q[0], {params[6]} | Rx q[1], {params[7]}}}
{{Rz q[0], {params[8]} | Rz q[1], {params[9]}}}

.measurement
measure_all
'''
    backend_type = qi.get_backend_type_by_name(backend)
    result = %time qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=shots)
    
    if result.get('histogram', {}):
        hist = result['histogram']
        expval = hist.get('0') + hist.get('1') - hist.get('2') - hist.get('3')
        return expval
    else:
        reason = result.get('raw_text', 'No reason in result structure.')
        raise RuntimeError(f'Result structure does not contain proper histogram data. {reason}')

In [25]:
from scipy.optimize import minimize

def find_minimum(params, backend):
    return minimize(lambda p: estimate_hamiltonian(p, backend=backend), params, method='nelder-mead',
                    options={'xtol': 1e-12, 'disp': True, 'maxiter': 10})

## Spin-2 back-end

In [26]:
np.random.seed(SEED)
params = np.random.uniform(low=-np.pi, high=np.pi, size=(10,))
find_minimum(params, 'Spin-2')

Running quantum circuit with params [ 0.82529653  1.58317926 -0.62257225 -2.7976498  -2.5527445  -0.54502968
  2.19602218 -0.99126826  0.88024056 -0.43346593]
CPU times: user 216 ms, sys: 9.31 ms, total: 226 ms
Wall time: 28.5 s
Running quantum circuit with params [ 0.86656135  1.58317926 -0.62257225 -2.7976498  -2.5527445  -0.54502968
  2.19602218 -0.99126826  0.88024056 -0.43346593]
CPU times: user 225 ms, sys: 27.2 ms, total: 252 ms
Wall time: 27 s
Running quantum circuit with params [ 0.82529653  1.66233823 -0.62257225 -2.7976498  -2.5527445  -0.54502968
  2.19602218 -0.99126826  0.88024056 -0.43346593]
CPU times: user 206 ms, sys: 5.49 ms, total: 212 ms
Wall time: 26.9 s
Running quantum circuit with params [ 0.82529653  1.58317926 -0.65370086 -2.7976498  -2.5527445  -0.54502968
  2.19602218 -0.99126826  0.88024056 -0.43346593]
CPU times: user 188 ms, sys: 22.7 ms, total: 211 ms
Wall time: 26.3 s
Running quantum circuit with params [ 0.82529653  1.58317926 -0.62257225 -2.93753229 -

 final_simplex: (array([[ 0.82529653,  1.58317926, -0.62257225, -2.7976498 , -2.5527445 ,
        -0.57228117,  2.19602218, -0.99126826,  0.88024056, -0.43346593],
       [ 0.83805231,  1.60764888, -0.63219472, -2.76703232, -2.51986515,
        -0.55345366,  2.2299639 , -1.00658931,  0.84103112, -0.43787688],
       [ 0.8371808 ,  1.60597705, -0.63153729, -2.67007697, -2.52211157,
        -0.55287811,  2.2276449 , -1.00554253,  0.89291602, -0.43017159],
       [ 0.84370721,  1.61849678, -0.59910623, -2.73243313, -2.55208937,
        -0.55718819,  2.18703599, -1.01338145,  0.87199091, -0.4391808 ],
       [ 0.83376129,  1.59941736, -0.62895775, -2.71998883, -2.65856291,
        -0.55061986,  2.17023351, -1.00143535,  0.86603052, -0.4346161 ],
       [ 0.86656135,  1.58317926, -0.62257225, -2.7976498 , -2.5527445 ,
        -0.54502968,  2.19602218, -0.99126826,  0.88024056, -0.43346593],
       [ 0.84453713,  1.62008881, -0.62339003, -2.78357908, -2.55931024,
        -0.55773627,  2.1776

## QX single-node simulator

In [12]:
np.random.seed(SEED)
params = np.random.uniform(low=-np.pi, high=np.pi, size=(10,))
%time find_minimum(params, 'QX single-node simulator')

Running quantum circuit with params [ 1.84702124  2.13986221  1.98050687  3.08476031  0.48552579  1.97145568
 -0.49437436 -2.96913204 -0.28816797 -2.47980934]


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.
Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


CPU times: user 232 ms, sys: 25.8 ms, total: 258 ms
Wall time: 27.2 s
Running quantum circuit with params [ 1.9393723   2.13986221  1.98050687  3.08476031  0.48552579  1.97145568
 -0.49437436 -2.96913204 -0.28816797 -2.47980934]
CPU times: user 195 ms, sys: 15.7 ms, total: 211 ms
Wall time: 22.2 s


RuntimeError: Result structure does not contain proper histogram data. Error occurred in simulation:
b"qx-simulator: ../nptl/pthread_mutex_lock.c:81: __pthread_mutex_cond_lock: Assertion `mutex->__data.__owner == 0' failed.\n"

## QX single-node SurfSara

In [13]:
np.random.seed(SEED)
params = np.random.uniform(low=-np.pi, high=np.pi, size=(10,))
%time find_minimum(params, 'QX single-node simulator SurfSara')

Running quantum circuit with params [ 1.99315243  1.24236     0.41020027 -1.41857174  3.1320029  -2.27424885
  0.72517815 -0.09401969 -0.59673273  1.4316181 ]


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


CPU times: user 4.16 s, sys: 283 ms, total: 4.44 s
Wall time: 8min 51s
Running quantum circuit with params [ 2.09281005  1.24236     0.41020027 -1.41857174  3.1320029  -2.27424885
  0.72517815 -0.09401969 -0.59673273  1.4316181 ]


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


CPU times: user 4.09 s, sys: 349 ms, total: 4.44 s
Wall time: 8min 44s
Running quantum circuit with params [ 1.99315243  1.304478    0.41020027 -1.41857174  3.1320029  -2.27424885
  0.72517815 -0.09401969 -0.59673273  1.4316181 ]


Your experiment can not be optimized and may take longer to execute, see https://www.quantum-inspire.com/kbase/optimization-of-simulations/ for details.


KeyboardInterrupt: 

AttributeError: 'NoneType' object has no attribute 'get'